In [184]:
import numpy as np
import copy
import math

First define all variables:

In [185]:
# Setting size of the network
j = 2 # number of junctions
z = 3 # number of links per junction
i = 2 # number of inflowing links per junction, has to be smaller than z
o = 1 # number of outflow links

# Inflowing links
I = np.ones((j,z)) # placeholder for inflowing links matrix
I[:,i:z] = 0 # set to zero for outflowing links per junction (assumes inflowing links have no outflowing traffic)

# Outflowing links
O = np.zeros((j,z)) # placeholder for inflowing links matrix
O[:,z-o:o] = 1 # set to zero for outflowing links per junction (assumes inflowing links have no outflowing traffic)

# Saturation flows
S = np.ones((j,z)) # Saturation flow for each incoming link

# Adjacency matrix
N = np.zeros((j*z,j*z)) # placeholder for junction adjacency
connections = [(z-1,z)] # only connection in this setup is from junction 1 link 3 to junction 2 link 1 (excluding vice versa)
for each in connections:
    N[each] = 1 # apply connections in adjacency matrix

# Setting time frame of the network
C = np.full((j),2) # cycle time
L = np.full((j),0.5) # lost time
T = 10 # control cycle

# Setting stage numbers 
i_s = 2 # number of stages per inflowing link
i_g = 1 # number of stages where inflowing link has right of way
F = np.zeros((j,z,i_s)) # matrix of stages per link
F[:,:,i_g] = 1 # setting green stages per link

# Setting green times and turning rates
G = np.ones((j,i)) # green time matrix
Tu = np.ones((j,i,o)) # turning rate matrix
for junction in range(j):
    G[junction,:] = G[junction,:] * np.random.random((i)) # assign random green times at each junction's incoming links
    Tu[junction,:] = Tu[junction,:] * np.random.random((i,o)) # assign a random turning rate to each junction's incoming/outflowing link combination
    Tu[junction,:] = 1 / np.sum(Tu[junction,:]) * Tu[junction,:] # making sure total turning rates are not larger than 1
Tu0 = np.ones(j) # turners who leave the system
Tu0 = Tu0*0.5 # balance with inflow
    
# demand and exit flows per junction
d = np.ones(j)*1.5 # steady-state demand flow: balances with turners who leave system

Now define the system matrices

In [186]:
# System matrices
A1 = np.zeros(j) # initialise one A1 for each junction
A2 = np.zeros(j) # initialise one A1 for each junction
A3 = np.zeros(j) # initialise one A1 for each junction
for junction in range(j):
    A1[junction] = (1-Tu0[junction]) * (Tu[junction,0,0] * S[junction,0] / C[junction])
    A2[junction] = (1-Tu0[junction]) * (Tu[junction,1,0] * S[junction,1] / C[junction])
    A3[junction] = (1-Tu0[junction]) * (S[junction,2] / C[junction])
    
X = np.full((j*o),10)  # initialise number of vehicles per outflow link

Introduce the dynamics

In [187]:
# Dynamics
X = X + T * (A1 * G[:,0] + A2 * G[:,1]  + A3 * np.array([G[0,0],1])) # when system is scaled up need to read third multiplier from adjacency matrix

Questions pending: How do we calculate the steady state green rates?